# RQ1: Feasibility Study

---

This notebook performs the training, hyperparameter optimization, and evaluation of the feasibility neural network model for predicting the most comparison-efficient sorting algorithm.

The **input file** is `dataset_training_200.csv` (containing presortedness features and labels).

---

## Requirements

In [ ]:
%pip install scikit-learn==1.7.0
%pip install pandas==2.2.3
%pip install numpy==1.26.0
%pip install tensorflow==2.15.0
%pip install matplotlib==3.10.1

## Train and Evaluate Neural Network Model

1. Grid search
2. Algorithm Prediction Summary: Compares the **actual vs. predicted counts** for each sorting algorithm in the test set and reports the number of **true positives** per class.

In [ ]:
import pandas as pd

from intellisorts_model_training import grid_search

# load training dataset
df_results = pd.read_csv('dataset_training_200.csv')

# features
train_input = df_results[['Inversions', 'Deletions', 'Runs', 'Dis']]
train_output = df_results['Algorithm']

# perform grid search
param_grid = {
    'batch_size': [512],
    'epochs': [500],
    'layers': [0,1,2,3,4,5,6,7,8,9],
    'layersize': [1,2,3,4,5,6,7,8,9,10]
}

(
    best_model,
    scaler,
    label_encoder,
    test_accuracy,
    test_indices,
    test_true_algorithms,
    test_predicted_algorithms
) = grid_search(
    'model_rq1',
    train_input,
    train_output,
    param_grid
)


---

## Average Comparisons Analysis

This section evaluates the **average number of comparisons** required by:

- The **minimum possible**, which represents the theoretical lower bound always choosing the most comparison efficient sorting algorithm
- The **prediction model**
- The classical sorting algorithms (Timsort, Merge Sort, Insertion Sort, Introsort, Quick Sort)

The results are visualized on a logarithmic scale to account for big differences in magnitude

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

test_set_df = df_results.iloc[test_indices]
test_set_df.reset_index(drop=True, inplace=True)

c = []
t = []

for index, row in test_set_df.iterrows():
    c.append(row[test_predicted_algorithms[index]])
    t.append(row[test_true_algorithms[index]])

def calculate_average(column_name):
    return round(np.sum(test_set_df[column_name]) / len(test_set_df), 1)

minimum_possible = round(np.sum(t)/len(test_set_df), 1)
algorithm_prediction_model = round(np.sum(c)/len(test_set_df), 1)

merge_sort_avg = calculate_average('merge_sort')
timsort_avg = calculate_average('timsort')
introsort_avg = calculate_average('introsort')
quick_sort_avg = calculate_average('quick_sort')
insertion_sort_avg = calculate_average('insertion_sort')

data = {
    'Algorithm': ['Optimal', 'Predicted','Timsort', 'Mergesort', 'Insertionsort', 'Introsort', 'Quicksort'],
    'Average Value': [minimum_possible, round(algorithm_prediction_model, 1), timsort_avg, merge_sort_avg, insertion_sort_avg, introsort_avg, quick_sort_avg]
}
df_average_comp = pd.DataFrame(data)

# Plot
plt.figure(figsize=(8, 5))
plt.bar(df_average_comp['Algorithm'], df_average_comp['Average Value'], color='skyblue')
plt.title('Average Comparisons of Sorting Algorithms')

for i, val in enumerate(df_average_comp['Average Value']):
    plt.text(i, val + 100, str(val), ha='center')

plt.ylabel('Average Comparisons (Logarithmic Scale)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--')
plt.yscale('log')
plt.tight_layout()
plt.savefig("figure_rq1_avg_comparisons.svg", format='svg')
plt.show()

print('Optimal: ', minimum_possible)
print('Predicted: ', algorithm_prediction_model)
print()
print('Mergesort: ', merge_sort_avg)
print('Timsort: ', timsort_avg)
print('Introsort: ', introsort_avg)
print('Quicksort: ', quick_sort_avg)
print('Insertionsort: ', insertion_sort_avg)